In [ ]:
# This script processes the 1997 wave of the Panel Study of Income Dynamics (PSID) 
# Child Development Supplement (CDS-I) data. The process is designed to be analogous 
# to the processing for the 2002 and 2007 waves and follows five main steps:
#
#   1. Merge all core 1997 CDS files and the Parent Identification File (PID)
#      to create a single cross-sectional dataset with permanent IDs.
#   2. Merge the 1997 PSID Family File to enrich the 1997 baseline.
#   3. Merge the longitudinal Transition to Adulthood (TAS) waves (2005 and 2015).
#   4. Process all 1997 Time Diary data to create both aggregate and contextual variables.
#   5. Perform the final merge to combine all data sources into a single analysis file.

import pandas as pd
import os

# --- Configuration: Define all base paths for the 1997 data processing ---
BASE_DATA_PATH = r'C:\Users\joshu\Aussie\Monash\Parental\Data'
CDS_1997_PATH = os.path.join(BASE_DATA_PATH, 'Supplemental Studies', 'Child Development Survey', 'CDS1997', '1997')
TAS_PATH = os.path.join(BASE_DATA_PATH, 'Supplemental Studies', 'Transition into Adulthood Supplement')
FAMILY_FILES_PATH = os.path.join(BASE_DATA_PATH, 'Main Study', 'Family Files')
# Path for the master Parent Identification File, used to get permanent longitudinal IDs.
PID_FILE_PATH = os.path.join(BASE_DATA_PATH, 'Main Study', 'Parent Identification 2023') 
ANALYSIS_PATH = os.path.join(BASE_DATA_PATH, 'Processed Data 1997')

# --- Helper Function to Load Data ---
def load_data(file_path, required=True):
    """Safely loads a CSV file, printing its status and shape."""
    try:
        df = pd.read_csv(file_path, low_memory=False)
        print(f"  - Successfully loaded: {os.path.basename(file_path)} (Shape: {df.shape})")
        return df
    except FileNotFoundError:
        if required:
            print(f"  - FATAL ERROR: Required file not found at {file_path}")
            raise
        else:
            print(f"  - Warning: Optional file not found, skipping: {os.path.basename(file_path)}")
            return None

# --- STEP 1: Merge Core CDS-I Data (1997 Wave) ---
def merge_core_cds_data_1997():
    """
    Loads and merges all raw 1997 CDS files into a single cross-sectional dataset.
    This function also merges the master Parent Identification File (PID) to append
    the permanent 1968 Family ID and Person Number, which are required for longitudinal merging.
    """
    print("\n--- Step 1: Merging Core CDS-I Data (1997 Wave) ---")
    
    # Load all necessary raw files for 1997
    demog_df = load_data(os.path.join(CDS_1997_PATH, 'DEMOG1997.csv'))
    pcg_chld_df = load_data(os.path.join(CDS_1997_PATH, 'PCG97_CHLD.csv'))
    child_df = load_data(os.path.join(CDS_1997_PATH, 'CHILD97.csv'), required=False)
    ocg_chld_df = load_data(os.path.join(CDS_1997_PATH, 'OCG_CHLD97.csv'), required=False)
    idmap_df = load_data(os.path.join(CDS_1997_PATH, 'IDMAP97.csv'))
    pcg_hhld_df = load_data(os.path.join(CDS_1997_PATH, 'PCG97_HH.csv'))
    # Load the Parent Identification File to get permanent IDs, replacing the need for GENMAP97.
    pid_df = load_data(os.path.join(PID_FILE_PATH, 'PID23.csv'))

    # Prepare base dataframe by standardizing the 1997 child identifiers.
    demog_df.rename(columns={'DEMID97': 'ID_1997', 'DEMSN97': 'SN_1997'}, inplace=True)
    merged_df = demog_df
    
    # Prepare and merge the PID file to add permanent identifiers (ER30001, ER30002).
    # PID4 is the 1997 Family ID and PID5 is the 1997 Person Number.
    pid_subset = pid_df[['PID2', 'PID3', 'PID4', 'PID5']].copy()
    pid_subset.rename(columns={'PID2': 'ER30001', 'PID3': 'ER30002', 'PID4': 'ID_1997', 'PID5': 'SN_1997'}, inplace=True)
    merged_df = pd.merge(merged_df, pid_subset, on=['ID_1997', 'SN_1997'], how='left')
    
    # Sequentially merge all child-level files.
    child_files = {'pcg_chld97': pcg_chld_df, 'child97': child_df, 'ocg_chld97': ocg_chld_df}
    key_map = {
        'pcg_chld97': ('PCGCHID97', 'PCGCHSN97'), 
        'child97': ('CHLDID97', 'CHLDSN97'),
        'ocg_chld97': ('OCGCID97', 'OCGCSN97')
    }
    
    for name, df in child_files.items():
        if df is not None:
            key_id, key_sn = key_map[name]
            if key_id in df.columns and key_sn in df.columns:
                df.rename(columns={key_id: 'ID_1997', key_sn: 'SN_1997'}, inplace=True)
                merged_df = pd.merge(merged_df, df, on=['ID_1997', 'SN_1997'], how='left', suffixes=('', f'_{name}'))
            else:
                print(f"  - Warning: Key columns {key_id}, {key_sn} not found in {name}. Skipping merge.")

    # Merge household data via the IDMAP file to link PCG-reported household characteristics.
    # The correct identifiers from the raw PCG97_HH.csv file are 'HHID97' and 'HHSN97'.
    if 'HHID97' in pcg_hhld_df.columns and 'HHSN97' in pcg_hhld_df.columns:
        pcg_hhld_df.rename(columns={'HHID97': 'PCGID97', 'HHSN97': 'PCGSN97'}, inplace=True)
    
        idmap_df.rename(columns={'CHILDID97': 'ID_1997', 'CHILDSN97': 'SN_1997'}, inplace=True)
        
        # Merge the ID map first to bring PCG keys into the main dataframe.
        merged_df = pd.merge(merged_df, idmap_df, on=['ID_1997', 'SN_1997'], how='left')
        # Now merge the PCG household data using the newly added PCG keys.
        merged_df = pd.merge(merged_df, pcg_hhld_df, on=['PCGID97', 'PCGSN97'], how='left', suffixes=('', '_pcghh97'))
    else:
        print("  - Warning: PCG Household identifiers not found in PCG97_HH.csv. Skipping merge.")

    # Remove any duplicated columns that may have arisen from merges.
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    print("Core 1997 CDS merge complete.")
    return merged_df

# --- STEP 2: Merge 1997 PSID Family File ---
def merge_family_file_1997(base_df):
    """
    Enriches the core CDS dataset by merging the 1997 PSID Family File. This adds
    a wide range of household-level economic and social variables.
    """
    print("\n--- Step 2: Merging 1997 PSID Family File ---")
    family_df = load_data(os.path.join(FAMILY_FILES_PATH, 'fam1997er', 'FAM1997ER.csv'))
    
    # The key in the 1997 family file is 'ER10002', which corresponds to the 1997 Interview Number.
    family_df.rename(columns={'ER10002': 'ID_1997'}, inplace=True)
    
    enriched_df = pd.merge(base_df, family_df, on='ID_1997', how='left', suffixes=('', '_fam1997'))
    
    enriched_df = enriched_df.loc[:, ~enriched_df.columns.duplicated()]
    print("1997 Family file merge complete.")
    return enriched_df

# --- STEP 3: Merge Longitudinal TAS Data ---
def merge_longitudinal_tas_data(base_df):
    """
    Merges the Transition to Adulthood (TAS) waves onto the base 1997 CDS dataset,
    linking children to their later-life outcomes using the permanent PSID identifiers.
    """
    print("\n--- Step 3: Merging Longitudinal TAS Data ---")
    
    tas_files = {
        '05': (os.path.join(TAS_PATH, 'ta2005', 'TA2005.csv'), 'TA050004', 'TA050005'),
        '15': (os.path.join(TAS_PATH, 'ta2015', 'TA2015.csv'), 'TA150004', 'TA150005')
    }
    
    longitudinal_df = base_df.copy()
    # Ensure the permanent IDs are present before attempting the merge.
    if 'ER30001' not in longitudinal_df.columns or 'ER30002' not in longitudinal_df.columns:
        print("  - Warning: Permanent IDs (ER30001, ER30002) not found. Cannot merge TAS data.")
        return longitudinal_df

    for year, (path, id_col, pn_col) in tas_files.items():
        tas_df = load_data(path, required=False)
        if tas_df is not None:
            tas_df.rename(columns={id_col: 'ER30001', pn_col: 'ER30002'}, inplace=True)
            longitudinal_df = pd.merge(longitudinal_df, tas_df, on=['ER30001', 'ER30002'], how='left', suffixes=('', f'_tas{year}'))
    
    longitudinal_df = longitudinal_df.loc[:, ~longitudinal_df.columns.duplicated()]
    print("TAS merge complete.")
    return longitudinal_df

# --- STEP 4: Process 1997 Time Diary Data ---
def process_time_diaries_1997():
    """
    Creates a standalone DataFrame with comprehensive time-use variables for the 1997 wave,
    including both aggregate weekly hours and specific measures of parental investment.
    """
    print("\n--- Step 4: Processing 1997 Time Diary Data ---")
    
    # Load both aggregate and raw activity time diary files.
    td_agg_df = load_data(os.path.join(CDS_1997_PATH, 'TD97_ACT_AGG.csv'), required=False) 
    td_activity_df = load_data(os.path.join(CDS_1997_PATH, 'TD97.csv'))

    # Part A: Calculate aggregate weekly hours if the aggregate file is available.
    part_a_df = None
    if td_agg_df is not None:
        child_identifiers_agg = td_agg_df[['AGGRID97', 'AGGRSN97']].copy().rename(columns={'AGGRID97': 'ID_1997', 'AGGRSN97': 'SN_1997'})
        part_a_df = calculate_aggregate_weekly_hours_1997(child_identifiers_agg, td_agg_df)
    
    # Part B: Calculate intensive parenting time from the raw activity file.
    part_b_df = calculate_intensive_parenting_time_1997(td_activity_df)
    
    # Combine the two parts into a single time-use dataset.
    if part_a_df is not None:
        time_use_df = pd.merge(part_a_df, part_b_df, on=['ID_1997', 'SN_1997'], how='outer')
    else:
        time_use_df = part_b_df

    time_use_df.fillna(0, inplace=True)
    print("1997 Time Diary processing complete.")
    return time_use_df

def calculate_aggregate_weekly_hours_1997(base_df, td_agg_df):
    """Calculates weekly average hours for 39 broad activity categories for 1997."""
    td_agg_df.rename(columns={'AGGRID97': 'ID_1997', 'AGGRSN97': 'SN_1997'}, inplace=True)
    panel_with_agg = pd.merge(base_df, td_agg_df, on=['ID_1997', 'SN_1997'], how='left')
    activity_codes = [f'39{i:02d}' for i in range(1, 40)]
    for code in activity_codes:
        wd_col, we_col = f'WD97{code}', f'WE97{code}'
        new_col = f'weekly_avg_hrs_cat_{code}_97'
        if wd_col in panel_with_agg.columns and we_col in panel_with_agg.columns:
            wd_sec = panel_with_agg[wd_col].fillna(0)
            we_sec = panel_with_agg[we_col].fillna(0)
            panel_with_agg[new_col] = ((wd_sec * 5) + (we_sec * 2)) / 3600
    new_cols = ['ID_1997', 'SN_1997'] + [col for col in panel_with_agg.columns if col.startswith('weekly_avg_hrs_cat_')]
    return panel_with_agg[new_cols]

def calculate_intensive_parenting_time_1997(td_activity_df):
    """Processes the raw 1997 activity file to calculate 'intensive parenting' measures."""
    # Define activity codes associated with skill-building interactions.
    skill_codes = [5490, 5491, 5492, 5493, 5494, 8010, 8011, 8012, 5040, 8020, 8030, 8040, 8090, 8510, 8520, 8211, 8212, 8213, 8214, 8215, 8221, 8222, 8223]
    
    # Column names from the raw TD97.csv file header.
    activity_col = 'COLA'
    day_of_week_col = 'T1'
    duration_col = 'DURATION'
    mother_col = 'COLG_B'
    father_col = 'COLG_C'

    skill_df = td_activity_df[td_activity_df[activity_col].isin(skill_codes)].copy()
    
    # Separate diaries into weekday (<=5) and weekend (>5).
    wd_skill_df = skill_df[skill_df[day_of_week_col] <= 5]
    we_skill_df = skill_df[skill_df[day_of_week_col] > 5]
    
    child_ids = td_activity_df[['TDID97', 'TDSN97']].drop_duplicates().rename(columns={'TDID97': 'ID_1997', 'TDSN97': 'SN_1997'})

    # Calculate time spent with each parent for both diary types.
    for day_type, df in [('wd', wd_skill_df), ('we', we_skill_df)]:
        for parent, col in [('mother', mother_col), ('father', father_col)]:
            mask = df[col] == 1
            time = df[mask].groupby(['TDID97', 'TDSN97'])[duration_col].sum().reset_index()
            time.rename(columns={duration_col: f'{parent}_interactive_{day_type}_sec_97', 'TDID97': 'ID_1997', 'TDSN97': 'SN_1997'}, inplace=True)
            child_ids = pd.merge(child_ids, time, on=['ID_1997', 'SN_1997'], how='left')

    # Ensure all columns exist and fill NaNs with 0.
    cols_to_fill = [f'{p}_interactive_{d}_sec_97' for p in ['mother', 'father'] for d in ['wd', 'we']]
    for col in cols_to_fill:
        if col not in child_ids.columns: 
            child_ids[col] = 0
        else: 
            child_ids[col] = child_ids[col].fillna(0)
            
    # Calculate total weekly hours from daily seconds, weighting for weekdays and weekends.
    child_ids['parent_interactive_skill_hrs_wk_97'] = \
        (((child_ids.get('mother_interactive_wd_sec_97', 0) + child_ids.get('father_interactive_wd_sec_97', 0)) * 5) +
         ((child_ids.get('mother_interactive_we_sec_97', 0) + child_ids.get('father_interactive_we_sec_97', 0)) * 2)) / 3600
         
    return child_ids[['ID_1997', 'SN_1997', 'parent_interactive_skill_hrs_wk_97']]

# --- Main Execution Block ---
if __name__ == '__main__':
    # Create the analysis folder if it doesn't exist.
    if not os.path.exists(ANALYSIS_PATH):
        os.makedirs(ANALYSIS_PATH)

    # Step 1: Merge core CDS data
    core_cds_df = merge_core_cds_data_1997()
    path_step1 = os.path.join(ANALYSIS_PATH, '01_cds_merged_1997.csv')
    core_cds_df.to_csv(path_step1, index=False)
    print(f"Step 1 intermediate file saved to: {path_step1}")

    # Step 2: Merge PSID Family File
    cds_family_df = merge_family_file_1997(core_cds_df)
    path_step2 = os.path.join(ANALYSIS_PATH, '02_cds_with_family_data_1997.csv')
    cds_family_df.to_csv(path_step2, index=False)
    print(f"Step 2 intermediate file saved to: {path_step2}")

    # Step 3: Merge longitudinal TAS data
    if 'ER30001' in cds_family_df.columns:
        cds_tas_panel = merge_longitudinal_tas_data(cds_family_df)
        path_step3 = os.path.join(ANALYSIS_PATH, '03_cds_tas_panel_1997.csv')
        cds_tas_panel.to_csv(path_step3, index=False)
        print(f"Step 3 intermediate file saved to: {path_step3}")
    else:
        print("\nSkipping Step 3: TAS Merge, because permanent IDs were not found.")
        cds_tas_panel = cds_family_df 

    # Step 4: Process Time Diary data
    time_use_variables = process_time_diaries_1997()
    path_step4 = os.path.join(ANALYSIS_PATH, '04_time_use_variables_1997.csv')
    time_use_variables.to_csv(path_step4, index=False)
    print(f"Step 4 intermediate file saved to: {path_step4}")

    # Step 5: Final Merge
    print("\n--- Step 5: Final Merge ---")
    final_dataset = pd.merge(cds_tas_panel, time_use_variables, on=['ID_1997', 'SN_1997'], how='left')
    final_dataset = final_dataset.loc[:, ~final_dataset.columns.duplicated()]
    print("All 1997 data sources successfully merged.")

    # Save final and sample outputs
    final_path = os.path.join(ANALYSIS_PATH, 'final_analysis_dataset_1997.csv')
    final_dataset.to_csv(final_path, index=False)
    print(f"Final dataset saved to: {final_path}")
    
    if len(final_dataset) >= 1000:
        sample_df = final_dataset.sample(n=1000, random_state=42)
        sample_path = os.path.join(ANALYSIS_PATH, 'sample_final_analysis_dataset_1997.csv')
        sample_df.to_csv(sample_path, index=False)
        print(f"Sample dataset saved to: {sample_path}")




--- Step 1: Merging Core CDS-I Data (1997 Wave) ---
  - Successfully loaded: DEMOG1997.csv (Shape: (3563, 27))
  - Successfully loaded: PCG97_CHLD.csv (Shape: (3563, 806))
  - Successfully loaded: CHILD97.csv (Shape: (2223, 339))
  - Successfully loaded: OCG_CHLD97.csv (Shape: (1395, 126))
  - Successfully loaded: IDMAP97.csv (Shape: (2233, 6))
  - Successfully loaded: PCG97_HH.csv (Shape: (1536, 311))
  - Successfully loaded: PID23.csv (Shape: (103725, 40))
Core 1997 CDS merge complete.
